<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%94%D0%97_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [5]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

Дані про банани

In [6]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/banana_quality.csv')

df['Quality'] = df['Quality'].map({'Good': 1, 'Bad': 0})

columns = df.columns.delete(-1)
data = df[columns]
df[columns] = (data - data.min()) / (data.max() - data.min())

In [7]:
df.head()

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,0.380309,0.626747,0.680712,0.360975,0.567312,0.671933,0.543416,1
1,0.349951,0.628093,0.485275,0.293682,0.481689,0.646850,0.545720,1
2,0.478460,0.699448,0.572694,0.283811,0.499358,0.716580,0.617337,1
3,0.446465,0.705394,0.595677,0.374028,0.473461,0.633590,0.556625,1
4,0.541672,0.687704,0.458841,0.378242,0.442847,0.579430,0.705908,1


Потрібно на основі інформації про банан спрогнозувати його якість `Quality`

# Завдання 1
Створіть клас Dataset для тренувальних та тестових даних

In [8]:
class BananaDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Завдання 2
Створіть DataLoader

In [9]:
train_size = int(0.8 * len(df))
test_size = len(df) - train_size

data_tensor = torch.tensor(df[columns].values, dtype=torch.float32)
labels_tensor = torch.tensor(df['Quality'].values, dtype=torch.float32)

dataset = BananaDataset(data_tensor, labels_tensor)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Завдання 3
Створіть нейромережу

In [10]:
class BananaNet(nn.Module):
    def __init__(self):
        super(BananaNet, self).__init__()
        self.fc1 = nn.Linear(len(columns), 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Завдання 4
Визначте функцію втрат та оптимізатор

In [11]:
model = BananaNet()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Завдання 5
Натренуйте нейромережу. Візміть невелику кількість епох

In [12]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.5366
Epoch [2/10], Loss: 0.4363
Epoch [3/10], Loss: 0.2861
Epoch [4/10], Loss: 0.1650
Epoch [5/10], Loss: 0.2619
Epoch [6/10], Loss: 0.1416
Epoch [7/10], Loss: 0.0898
Epoch [8/10], Loss: 0.1006
Epoch [9/10], Loss: 0.3676
Epoch [10/10], Loss: 0.1256


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [13]:
torch.save(model.state_dict(), 'banana_model.pth')

model.load_state_dict(torch.load('banana_model.pth'))
model.eval()

with torch.no_grad():
    sample_data = data_tensor[0].unsqueeze(0)
    prediction = model(sample_data).item()
    print(f'Prediction: {prediction:.4f}')

Prediction: 0.9986
